In [13]:
!pip install librosa

  Using cached librosa-0.10.1-py3-none-any.whl (253 kB)
     ---------------------------------------- 2.6/2.6 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 184.6/184.6 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 222.8/222.8 kB 2.7 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
     ---------------------------------------- 9.3/9.3 MB 3.6 MB/s eta 0:00:00
  Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
     ---------------------------------------- 44.1/44.1 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 28.1/28.1 MB 4.9 MB/s eta 0:00:00
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
import librosa
import csv

In [8]:
!pip install tensorflow==2.8 tensorflow-gpu==2.8 opencv-python matplotlib  protobuf==3.19.1


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip list

Package                      Version
---------------------------- -------------------
absl-py                      2.0.0
asttokens                    2.4.1
astunparse                   1.6.3
cachetools                   5.3.2
certifi                      2023.7.22
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.0
contourpy                    1.2.0
cycler                       0.12.1
debugpy                      1.8.0
decorator                    5.1.1
exceptiongroup               1.1.3
executing                    2.0.1
flatbuffers                  23.5.26
fonttools                    4.44.3
gast                         0.5.4
google-auth                  2.23.4
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.59.2
h5py                         3.10.0
idna                         3.4
ipykernel                    6.26.0
ipython                      8.17.2
jedi                        


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import tensorflow as tf

In [2]:
current_directory = os.getcwd()
print(current_directory)

D:\IVP_StressDetection


In [3]:
os.chdir('Dataset')

In [4]:
current_directory = os.getcwd()
print(current_directory)

D:\IVP_StressDetection\Dataset


In [160]:

def assign_index(filename, column_name):
    df = pd.read_csv(filename + '.csv')

    # Set a specific column as the index (replace 'column_name' with your column name)
    df.set_index(column_name, inplace=True)  # Set the desired column as index

    # Convert index values to strings
    df.index = df.index.astype(str)

    # Extract index values as a list of strings
    index_values = df.index.tolist()
    
    return index_values


In [161]:

def assign_label(filename, column_names):
    df = pd.read_csv(filename + '.csv')

    index_values = {}
    for col in column_names:
        try:
            # Convert column values to numeric type
            df[col] = pd.to_numeric(df[col])
            # Convert column values to strings based on condition
            df[col] = df[col].apply(lambda x: '1' if x >= 10 else '0')
            # Set each specified column as the index
            df.set_index(col, inplace=True)
            # Store index values in a dictionary
            index_values = df.index.tolist()
            # Reset the index after processing each column
        except ValueError:
            print(f"Error processing column '{col}'. Ensure the values are numeric.")

    return index_values




In [180]:
file_names_train = assign_index("train_split", "Participant_ID") 
file_names_test = assign_index("test_split", "Participant_ID")
file_names_val = assign_index("dev_split", "Participant_ID")

column_to_process = ['PHQ_Score']

labels_train = assign_label("train_split", column_to_process)
labels_test = assign_label("test_split", column_to_process)
labels_val = assign_label("dev_split", column_to_process)
print(file_names_test)

print(len(file_names_train))
print(len(file_names_test))
print(len(labels_train))
print(len(labels_test))


['600', '602', '604', '605', '606', '607', '615', '618', '619', '624', '626', '631', '636', '638', '650', '651', '652', '655', '656', '658', '663', '664', '676', '679', '682', '688', '693', '696', '699', '705', '708', '710', '712', '715', '716', '718']
153
36
153
36


In [181]:

# Create lists for file paths and label names
def openfile(file_names,labels):
    file_paths = []
    label_names = []

    for dire, label in zip(file_names,labels):
        # Create the full file path
        file_path = f"{dire}_P/{dire}_P/{dire}_AUDIO.wav"
        file_paths.append(file_path)

        # Convert label to distress or stress
        label_name = "distressed" if label == "0" else "stressed"
        label_names.append(label_name)

    # Create a DataFrame
    data_df = pd.DataFrame({'Path': file_paths, 'Label': label_names})

    # Display the DataFrame
    return data_df

In [182]:
train_data = openfile(file_names_train,labels_train)
test_data = openfile(file_names_test,labels_test)
val_data = openfile(file_names_val,labels_val)

In [183]:
print(test_data)

                         Path       Label
0   600_P/600_P/600_AUDIO.wav  distressed
1   602_P/602_P/602_AUDIO.wav    stressed
2   604_P/604_P/604_AUDIO.wav    stressed
3   605_P/605_P/605_AUDIO.wav  distressed
4   606_P/606_P/606_AUDIO.wav  distressed
5   607_P/607_P/607_AUDIO.wav  distressed
6   615_P/615_P/615_AUDIO.wav  distressed
7   618_P/618_P/618_AUDIO.wav  distressed
8   619_P/619_P/619_AUDIO.wav  distressed
9   624_P/624_P/624_AUDIO.wav    stressed
10  626_P/626_P/626_AUDIO.wav  distressed
11  631_P/631_P/631_AUDIO.wav  distressed
12  636_P/636_P/636_AUDIO.wav    stressed
13  638_P/638_P/638_AUDIO.wav    stressed
14  650_P/650_P/650_AUDIO.wav  distressed
15  651_P/651_P/651_AUDIO.wav  distressed
16  652_P/652_P/652_AUDIO.wav  distressed
17  655_P/655_P/655_AUDIO.wav    stressed
18  656_P/656_P/656_AUDIO.wav  distressed
19  658_P/658_P/658_AUDIO.wav    stressed
20  663_P/663_P/663_AUDIO.wav  distressed
21  664_P/664_P/664_AUDIO.wav  distressed
22  676_P/676_P/676_AUDIO.wav  dis

In [103]:
train_data['Label'].value_counts()

Label
distressed    107
stressed       46
Name: count, dtype: int64

In [184]:
def trimm(data):
    data_trimmed, _ = librosa.effects.trim(data, top_db=20)
    return data_trimmed

def pitch(data, samplingRate, pitch_factor):
    return librosa.effects.pitch_shift(y=data, sr=samplingRate, n_steps=pitch_factor)


In [185]:
# feature extraction (mfcc, delta, double-delta, chroma)

def extract_features(filename):
    with sf.SoundFile(filename) as soundFile:
        x = soundFile.read(dtype="float64")
        sampleRate = soundFile.samplerate
        res = np.array([])
        
        x = trimm(x)
        x = pitch(x, sampleRate, 3)
        
        stft = np.abs(librosa.stft(x))
        
        # for mfcc
        mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sampleRate, n_mfcc=13).T, axis=0)
        res = np.hstack((res, mfccs))
        
        # for delta
        delta_mfcc = librosa.feature.delta(mfccs)
        res = np.hstack((res, delta_mfcc))
        
        # for double-delta
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)
        res = np.hstack((res, delta2_mfcc))
        
        # for chroma
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampleRate).T, axis=0)
        res = np.hstack((res, chroma))
        
        # for mel spectrogram
        melSpec = np.mean(librosa.feature.melspectrogram(y=x, sr=sampleRate).T, axis=0)
        res = np.hstack((res, melSpec))
        
        # for contrast
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sampleRate).T, axis=0)
        res = np.hstack((res, contrast))
        
        # for tonnetz
        tonnetz = np.mean(librosa.feature.tonnetz(y=x, sr=sampleRate).T, axis=0)
        res = np.hstack((res, tonnetz))
        
    return res

In [188]:
from sklearn.model_selection import train_test_split
def loadData(df,test_size=0.15):
    x= []
    for ind in df.index:
        emotion = df['Label'][ind]
        file = df['Path'][ind] 
        a = file[:3]
        b=int(a)
        
        #f(b>631):
        print(file)
        feature = extract_features(file)
        x.append(feature)
    return np.array(x)

In [ ]:
import tarfile
i=0

# Path to the tar file
#for name in file_names_train:
    if(i<=15):
        i=i+1
        continue
    else:
        tar_file_path = name+"_P.tar"

        # Open the tar file
        with tarfile.open(tar_file_path, 'r') as tar_ref:
            # Extract all contents to a specific directory
            tar_ref.extractall(name+"_P")


In [81]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
    

In [82]:
tf.config.list_physical_devices('GPU')

[]

In [189]:
# X_train,X_test,y_train,y_test = loadData(data_df)

#X_train = loadData(train_data)
X_test = loadData(test_data)
#X_val = loadData(val_data)
#X_train.shape,X_test.shape

600_P/600_P/600_AUDIO.wav
602_P/602_P/602_AUDIO.wav
604_P/604_P/604_AUDIO.wav
605_P/605_P/605_AUDIO.wav
606_P/606_P/606_AUDIO.wav
607_P/607_P/607_AUDIO.wav
615_P/615_P/615_AUDIO.wav
618_P/618_P/618_AUDIO.wav
619_P/619_P/619_AUDIO.wav
624_P/624_P/624_AUDIO.wav
626_P/626_P/626_AUDIO.wav
631_P/631_P/631_AUDIO.wav
636_P/636_P/636_AUDIO.wav
638_P/638_P/638_AUDIO.wav
650_P/650_P/650_AUDIO.wav
651_P/651_P/651_AUDIO.wav
652_P/652_P/652_AUDIO.wav
655_P/655_P/655_AUDIO.wav
656_P/656_P/656_AUDIO.wav
658_P/658_P/658_AUDIO.wav
663_P/663_P/663_AUDIO.wav
664_P/664_P/664_AUDIO.wav
676_P/676_P/676_AUDIO.wav
679_P/679_P/679_AUDIO.wav
682_P/682_P/682_AUDIO.wav
688_P/688_P/688_AUDIO.wav
693_P/693_P/693_AUDIO.wav
696_P/696_P/696_AUDIO.wav
699_P/699_P/699_AUDIO.wav
705_P/705_P/705_AUDIO.wav
708_P/708_P/708_AUDIO.wav
710_P/710_P/710_AUDIO.wav
712_P/712_P/712_AUDIO.wav
715_P/715_P/715_AUDIO.wav
716_P/716_P/716_AUDIO.wav
718_P/718_P/718_AUDIO.wav


In [114]:
print(X_train)

NameError: name 'X_train' is not defined

In [190]:
#X_train_df = pd.DataFrame(X_train,columns = [f"feat_{i}" for i in range(192)])
X_test_df = pd.DataFrame(X_test,columns = [f"feat_{i}" for i in range(192)])
#X_val_df = pd.DataFrame(X_val,columns = [f"feat_{i}" for i in range(192)])

In [191]:
#X_train_df.to_csv('X_train.csv')
X_test_df.to_csv('X_test.csv')
#X_val_df.to_csv('X_val.csv')
# y_train_df.to_csv('y_train_imbalanced.csv')
# y_test_df.to_csv('y_test_imbalanced.csv')